In [1]:
import json
# read api key from tally_api_key.json
TALLY_API_KEY = json.load(open('tally_api_key.json'))['key']

In [2]:
import requests

url = "https://api.tally.xyz/query"

headers = {
  'Api-Key': TALLY_API_KEY,
  'Content-Type': 'application/json'
}

In [3]:
payload="{\"query\":\"query Chains {\\n    chains {\\n        id\\n    }\\n}\",\"variables\":{}}"

chains_response = requests.request("POST", url, headers=headers, data=payload)

chains_data = chains_response.json()
chains_data

{'data': {'chains': [{'id': 'eip155:534353'},
   {'id': 'eip155:1'},
   {'id': 'eip155:69'},
   {'id': 'eip155:100'},
   {'id': 'eip155:4'},
   {'id': 'eip155:421611'},
   {'id': 'eip155:97'},
   {'id': 'eip155:42'},
   {'id': 'eip155:137'},
   {'id': 'eip155:10'},
   {'id': 'eip155:84531'},
   {'id': 'eip155:56'},
   {'id': 'eip155:420'},
   {'id': 'eip155:1284'},
   {'id': 'eip155:42170'},
   {'id': 'eip155:11155111'},
   {'id': 'eip155:42161'},
   {'id': 'eip155:43113'},
   {'id': 'eip155:5'},
   {'id': 'eip155:412346'},
   {'id': 'eip155:421613'},
   {'id': 'eip155:43114'},
   {'id': 'eip155:80001'}]}}

In [4]:
chains = chains_data['data']['chains']
chains = [c['id'] for c in chains]
chains

['eip155:534353',
 'eip155:1',
 'eip155:69',
 'eip155:100',
 'eip155:4',
 'eip155:421611',
 'eip155:97',
 'eip155:42',
 'eip155:137',
 'eip155:10',
 'eip155:84531',
 'eip155:56',
 'eip155:420',
 'eip155:1284',
 'eip155:42170',
 'eip155:11155111',
 'eip155:42161',
 'eip155:43113',
 'eip155:5',
 'eip155:412346',
 'eip155:421613',
 'eip155:43114',
 'eip155:80001']

In [40]:
def make_deployments_query(chain: str) -> str:
    query = """query Proposals {
        proposals(
            chainId: "%s"
        ) {
            platform_deployment_id: governanceId
            proposal_id: id
            author: proposer {
                id
            }
            date: start {
                timestamp
            }
            voteStats {
                support
                votes
            }
        }
    }"""
    return query % chain

In [6]:
# make chainIds a string of comma separated values
chainIds = ', '.join(['"%s"' % c for c in chains])
query = """
query {
  governors(chainIds: [%s], includeInactive: true) {
    platform_id: id
    name
  }
}
""" % chainIds
response = requests.request("POST", url, headers=headers, data=json.dumps({'query': query}))
data = response.json()
data

{'data': {'governors': [{'platform_id': 'eip155:97:0x1fD1587342082e4579d786E3C15CbD9dc9Ff7B8f',
    'name': 'Gov4'},
   {'platform_id': 'eip155:80001:0xa6d84A580d0a565B85CC4Cd69EBCbb8b64F42ebA',
    'name': 'Carbon Standard DAO v6'},
   {'platform_id': 'eip155:5:0x64c7d9d710f938226f2335Cc6168cBb30621a57F',
    'name': 'SWEEPT DAO'},
   {'platform_id': 'eip155:80001:0x96eD2a6a27A3Cf718797aA212F595B95a4Ad2651',
    'name': 'SKINARMA'},
   {'platform_id': 'eip155:80001:0xEd825d4DE63ac4B969920CC3705907fD532ac8E3',
    'name': 'kjhnlojnlojnlojn'},
   {'platform_id': 'eip155:80001:0x7aA7d28eB17d3555cE8d0974F9006Fe670F53e1e',
    'name': 'TestDao2'},
   {'platform_id': 'eip155:80001:0x51d3b7abDb44c2564fb610C1bbc712a49E5aD756',
    'name': 'MyDAODemo'},
   {'platform_id': 'eip155:80001:0x3C560826Cc3f6aF36746043B4FCc03F920415072',
    'name': 'Doodle DAO'},
   {'platform_id': 'eip155:80001:0x9C955a246c3F2a6ce77d213f9c389A75b84a7351',
    'name': 'Mumbai Test'},
   {'platform_id': 'eip155:137:0x

In [9]:
len(data['data']['governors'])

2375

In [10]:
import pandas as pd
df = pd.DataFrame(data['data']['governors'])
df.head()

,platform_id,name
0,eip155:97:0x1fD1587342082e4579d786E3C15CbD9dc9...,Gov4
1,eip155:80001:0xa6d84A580d0a565B85CC4Cd69EBCbb8...,Carbon Standard DAO v6
2,eip155:5:0x64c7d9d710f938226f2335Cc6168cBb3062...,SWEEPT DAO
3,eip155:80001:0x96eD2a6a27A3Cf718797aA212F595B9...,SKINARMA
4,eip155:80001:0xEd825d4DE63ac4B969920CC3705907f...,kjhnlojnlojnlojn


In [11]:
# * `platform`
df['platform'] = 'tally'
# * `platform_id` no chg
# * `name` (may be nan) no chg
# * `website` (may be nan)
df['website'] = None
# * `additional` (may contain website, social, etc. info, may be nan)
df['additional'] = None
# * `votes_count` (the number of unique votes in the deployment)
# TKTK
# * `proposals_count` (the number of proposals in the deployment)
# TKTK
df.head()

,platform_id,name,platform,website,additional
0,eip155:97:0x1fD1587342082e4579d786E3C15CbD9dc9...,Gov4,tally,None,None
1,eip155:80001:0xa6d84A580d0a565B85CC4Cd69EBCbb8...,Carbon Standard DAO v6,tally,None,None
2,eip155:5:0x64c7d9d710f938226f2335Cc6168cBb3062...,SWEEPT DAO,tally,None,None
3,eip155:80001:0x96eD2a6a27A3Cf718797aA212F595B9...,SKINARMA,tally,None,None
4,eip155:80001:0xEd825d4DE63ac4B969920CC3705907f...,kjhnlojnlojnlojn,tally,None,None


In [20]:
proposals_df = pd.read_csv('../../proposals/tally/tally_proposals.csv')
proposals_df = proposals_df[['platform_deployment_id', 'proposal_id']]
proposals_df.head()

,platform_deployment_id,proposal_id
0,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,2469566067493070112382155014139101658401666661...
1,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,3687053733533920144921907414590262232442515184...
2,eip155:4:0x1874306110B9cd9CcE56Ba5748cb4f78DCA...,2374853609165189405470451842531233611721691995...
3,eip155:4:0xE108cdBc1eE0F775D93C12C071a1F97d94A...,1037135589049921000232560869388807622276297882...
4,eip155:4:0xB09525419AfF0c7e28c07597DA9EdEe2BdF...,1


In [21]:
# rename platform_deployment_id to platform_id
proposals_df = proposals_df.rename(columns={'platform_deployment_id': 'platform_id'})
# group by platform_id and count the number of proposals
proposals_df = proposals_df.groupby('platform_id').count().reset_index()
# rename proposal_id to proposals_count
proposals_df = proposals_df.rename(columns={'proposal_id': 'proposals_count'})
proposals_df.head()

,platform_id,proposals_count
0,eip155:100:0xeC61AbBecBEA81C2168f8AA8b658942D5...,1
1,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,1
2,eip155:10:0x3c842a744F21F5fA59F05C399f2E567854...,1
3,eip155:10:0x557bbcB62D1152eF76404c9f27166F7B08...,2
4,eip155:10:0x68AD9B946332eaB14A26945815b68Df04A...,5


In [22]:
# join
df = df.merge(proposals_df, on='platform_id', how='left', validate='one_to_one')
df.head()

,platform_id,name,platform,website,additional,proposals_count
0,eip155:97:0x1fD1587342082e4579d786E3C15CbD9dc9...,Gov4,tally,None,None,NaN
1,eip155:80001:0xa6d84A580d0a565B85CC4Cd69EBCbb8...,Carbon Standard DAO v6,tally,None,None,NaN
2,eip155:5:0x64c7d9d710f938226f2335Cc6168cBb3062...,SWEEPT DAO,tally,None,None,NaN
3,eip155:80001:0x96eD2a6a27A3Cf718797aA212F595B9...,SKINARMA,tally,None,None,NaN
4,eip155:80001:0xEd825d4DE63ac4B969920CC3705907f...,kjhnlojnlojnlojn,tally,None,None,NaN


In [23]:
# how many non-null proposals_count values are there?
df['proposals_count'].notnull().sum()

1763

In [24]:
# how many null proposals_count values are there?
df['proposals_count'].isnull().sum()

612

In [25]:
# fill nulls with 0
df['proposals_count'] = df['proposals_count'].fillna(0)
# make col an int
df['proposals_count'] = df['proposals_count'].astype(int)
df.head()

,platform_id,name,platform,website,additional,proposals_count
0,eip155:97:0x1fD1587342082e4579d786E3C15CbD9dc9...,Gov4,tally,None,None,0
1,eip155:80001:0xa6d84A580d0a565B85CC4Cd69EBCbb8...,Carbon Standard DAO v6,tally,None,None,0
2,eip155:5:0x64c7d9d710f938226f2335Cc6168cBb3062...,SWEEPT DAO,tally,None,None,0
3,eip155:80001:0x96eD2a6a27A3Cf718797aA212F595B9...,SKINARMA,tally,None,None,0
4,eip155:80001:0xEd825d4DE63ac4B969920CC3705907f...,kjhnlojnlojnlojn,tally,None,None,0


In [26]:
# now for votes
votes_df = pd.read_csv('../../votes/tally/tally_votes.csv')
votes_df = votes_df[['platform_deployment_id', 'voter',]]
votes_df.head()

,platform_deployment_id,voter
0,eip155:420:0x50E31E0A8004a8Fa5EAC619e0bfCD9F59...,eip155:420:0x3382Bb7214c109f12Ffe8aA9C39BAf7eD...
1,eip155:420:0xcDbeE6389C1215e659354465d8413438C...,eip155:420:0x5632aB6622614bc3eB4AeA5e04f431784...
2,eip155:420:0xcDbeE6389C1215e659354465d8413438C...,eip155:420:0x7021dc1C26b4c8ddB6c6846FBF4b8Fa56...
3,eip155:420:0xcDbeE6389C1215e659354465d8413438C...,eip155:420:0x64C83e16Ab9e31c1aCB3d8dDa626EC198...
4,eip155:420:0xcDbeE6389C1215e659354465d8413438C...,eip155:420:0x1b0a4D449bc0913B9A918702e31274AC3...


In [27]:
# group by platform_deployment_id and count the number of voters
votes_df = votes_df.groupby('platform_deployment_id').count().reset_index()
# rename voter to votes_count
votes_df = votes_df.rename(columns={'voter': 'votes_count'})
# rename platform_deployment_id to platform_id
votes_df = votes_df.rename(columns={'platform_deployment_id': 'platform_id'})
votes_df.head()

,platform_id,votes_count
0,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,1
1,eip155:10:0x68AD9B946332eaB14A26945815b68Df04A...,4
2,eip155:10:0x7d347cb7473dA7f64f9Ea940a6F9b5e1b3...,1
3,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,1
4,eip155:10:0x81e7fA0a7Fc2Ad0Fde1947d67aD7965038...,3


In [28]:
# join
df = df.merge(votes_df, on='platform_id', how='left', validate='one_to_one')
df.head()

,platform_id,name,platform,website,additional,proposals_count,votes_count
0,eip155:97:0x1fD1587342082e4579d786E3C15CbD9dc9...,Gov4,tally,None,None,0,NaN
1,eip155:80001:0xa6d84A580d0a565B85CC4Cd69EBCbb8...,Carbon Standard DAO v6,tally,None,None,0,NaN
2,eip155:5:0x64c7d9d710f938226f2335Cc6168cBb3062...,SWEEPT DAO,tally,None,None,0,NaN
3,eip155:80001:0x96eD2a6a27A3Cf718797aA212F595B9...,SKINARMA,tally,None,None,0,NaN
4,eip155:80001:0xEd825d4DE63ac4B969920CC3705907f...,kjhnlojnlojnlojn,tally,None,None,0,NaN


In [29]:
# how many nulls vs non nulls in votes_count?
df['votes_count'].isnull().sum(), df['votes_count'].notnull().sum()

(1045, 1330)

In [30]:
# fill nulls with 0
df['votes_count'] = df['votes_count'].fillna(0)
# make col an int
df['votes_count'] = df['votes_count'].astype(int)
df.head()

,platform_id,name,platform,website,additional,proposals_count,votes_count
0,eip155:97:0x1fD1587342082e4579d786E3C15CbD9dc9...,Gov4,tally,None,None,0,0
1,eip155:80001:0xa6d84A580d0a565B85CC4Cd69EBCbb8...,Carbon Standard DAO v6,tally,None,None,0,0
2,eip155:5:0x64c7d9d710f938226f2335Cc6168cBb3062...,SWEEPT DAO,tally,None,None,0,0
3,eip155:80001:0x96eD2a6a27A3Cf718797aA212F595B9...,SKINARMA,tally,None,None,0,0
4,eip155:80001:0xEd825d4DE63ac4B969920CC3705907f...,kjhnlojnlojnlojn,tally,None,None,0,0


In [31]:
# reorder cols to put votes_count before proposals_count
df = df[['platform', 'platform_id', 'name', 'website', 'additional', 'votes_count', 'proposals_count']]
df.head()

,platform,platform_id,name,website,additional,votes_count,proposals_count
0,tally,eip155:97:0x1fD1587342082e4579d786E3C15CbD9dc9...,Gov4,None,None,0,0
1,tally,eip155:80001:0xa6d84A580d0a565B85CC4Cd69EBCbb8...,Carbon Standard DAO v6,None,None,0,0
2,tally,eip155:5:0x64c7d9d710f938226f2335Cc6168cBb3062...,SWEEPT DAO,None,None,0,0
3,tally,eip155:80001:0x96eD2a6a27A3Cf718797aA212F595B9...,SKINARMA,None,None,0,0
4,tally,eip155:80001:0xEd825d4DE63ac4B969920CC3705907f...,kjhnlojnlojnlojn,None,None,0,0


In [32]:
# save to csv
df.to_csv('tally_deployments.csv', index=False)